# Next steps

Figure out how to display label for examples for validation code

In [25]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
from ogb.nodeproppred import NodePropPredDataset
import numpy as np
import tensorflow as tf
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from typing import Mapping
from tensorflow_gnn.experimental import sampler
from tensorflow_gnn.models import mt_albis
from tensorflow.keras.callbacks import TensorBoard
import functools
import itertools

In [2]:
dataset = NodePropPredDataset(name = "ogbn-products", root = 'dataset/')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
NUM_TRAINING_SAMPLES=train_idx.shape[0]
NUM_VALIDATION_SAMPLES = valid_idx.shape[0]

In [28]:
graph, label = dataset[0]
mask0=np.isin(graph["edge_index"][0],train_idx)
mask1=np.isin(graph["edge_index"][1],train_idx)

mask = mask0 & mask1
indices = np.where(mask)[0]
train_edge_index=graph["edge_index"][:, indices]
train_node_feat = graph["node_feat"][train_idx,:]
train_label = label[train_idx]

In [29]:
# Trying validation set here
graph, label = dataset[0]
mask0=np.isin(graph["edge_index"][0],valid_idx)
mask1=np.isin(graph["edge_index"][1],valid_idx)

mask = mask0 & mask1
indices = np.where(mask)[0]
valid_edge_index=graph["edge_index"][:, indices]
valid_node_feat = graph["node_feat"][valid_idx,:]
valid_label = label[valid_idx]

In [4]:
graph_schema = tfgnn.read_schema("graph_schema.pbtxt")

In [42]:
# graph_schema = tfgnn.read_schema("graph_schema.pbtxt")
# graph_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)
# train_dataset_provider = runner.TFRecordDatasetProvider(file_pattern="train.tfrecord")
# train_dataset = train_dataset_provider.get_dataset(context=tf.distribute.InputContext())
# train_dataset = train_dataset.map(lambda serialized: tfgnn.parse_single_example(serialized=serialized, spec=graph_spec))
# graph_tensor = train_dataset.get_single_element()

In [7]:
# graph_schema = tfgnn.read_schema("graph_schema.pbtxt")

graph, label = dataset[0]
SIZE=2449029

graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets={
       "product": tfgnn.NodeSet.from_fields(
           sizes = tf.constant([2449029]),
           features={
               "id": tf.range(0,SIZE),
               "feature": tf.constant(graph["node_feat"]),
               "label": tf.constant(label)
           }
       )
   },
    edge_sets = {
      "bought_together": tfgnn.EdgeSet.from_fields(
          sizes = tf.constant([graph["edge_index"].shape[1]]),
          adjacency = tfgnn.Adjacency.from_indices(
              source = ("product", tf.constant(graph["edge_index"][0,:])),
              target = ("product", tf.constant(graph["edge_index"][1,:])),
          )
      )  
    },
)

In [30]:
graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets={
       "product": tfgnn.NodeSet.from_fields(
           sizes = tf.constant([train_idx.shape[0]]),
           features={
               "id": tf.constant(train_idx),
               "feature": tf.constant(train_node_feat),
               "label": tf.constant(train_label)
           }
       )
   },
    edge_sets = {
      "bought_together": tfgnn.EdgeSet.from_fields(
          sizes = tf.constant([train_edge_index.shape[1]]),
          adjacency = tfgnn.Adjacency.from_indices(
              source = ("product", tf.constant(train_edge_index[0,:])),
              target = ("product", tf.constant(train_edge_index[1,:])),
          )
      )  
    },
)

In [31]:
valid_graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets={
       "product": tfgnn.NodeSet.from_fields(
           sizes = tf.constant([valid_idx.shape[0]]),
           features={
               "id": tf.range(196615, 39323+196615),
               "feature": tf.constant(valid_node_feat),
               "label": tf.constant(valid_label)
           }
       )
   },
    edge_sets = {
      "bought_together": tfgnn.EdgeSet.from_fields(
          sizes = tf.constant([valid_edge_index.shape[1]]),
          adjacency = tfgnn.Adjacency.from_indices(
              source = ("product", tf.constant(valid_edge_index[0,:])),
              target = ("product", tf.constant(valid_edge_index[1,:])),
          )
      )  
    },
)

In [8]:
train_sampling_sizes = {
    "bought_together": 8,
}

def create_sampling_model(full_graph_tensor: tfgnn.GraphTensor, sizes: Mapping[str, int]) -> tf.keras.Model:

    def edge_sampler(sampling_op: tfgnn.sampler.SamplingOp):    
        edge_set_name = sampling_op.edge_set_name
        sample_size = sizes[edge_set_name]
        return sampler.InMemUniformEdgesSampler.from_graph_tensor(
            full_graph_tensor,
            edge_set_name, sample_size=sample_size
        )
    def get_features(node_set_name: tfgnn.NodeSetName):
        return sampler.InMemIndexToFeaturesAccessor.from_graph_tensor(
            full_graph_tensor,
            node_set_name
        )

    sampling_spec_builder = tfgnn.sampler.SamplingSpecBuilder(graph_schema)
    seed = sampling_spec_builder.seed("product")
    products_bought_together = seed.sample(sizes["bought_together"], "bought_together", op_name="bt")
    sampling_spec = sampling_spec_builder.build()
    model = sampler.create_sampling_model_from_spec(graph_schema, sampling_spec, edge_sampler, get_features, seed_node_dtype=tf.int64)
    return model

In [45]:
# def node_sets_fn(node_set, *, node_set_name):
#     features = node_set.get_features_dict()
#     ids = features.pop('id')
#     num_bins = 50000
#     features['hashed_id'] = tf.cast(tf.keras.layers.Hashing(num_bins=num_bins)(ids), tf.int32)
#     return features
# graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=node_sets_fn)(graph_tensor)

In [9]:
class SubgraphDatasetProvider(runner.DatasetProvider):
    "Dataset provider"

    def __init__(self,
                full_graph_tensor: tfgnn.GraphTensor,
                sizes: Mapping[str, int],
                dataset: tf.data.Dataset):
        self._sampling_model = create_sampling_model(full_graph_tensor, sizes)
        self.input_graph_spec = self._sampling_model.output.spec
        self._seed_dataset = dataset
        
    def get_dataset(self, context: tf.distribute.InputContext) -> tf.data.Dataset:
        """Creates TF dataset"""
        ds = self._seed_dataset.shard(num_shards=context.num_input_pipelines, index = context.input_pipeline_id)
        ds = ds.shuffle(NUM_TRAINING_SAMPLES).repeat()
        ds = ds.batch(128)
        ds = ds.map(
            functools.partial(self.sample),
            num_parallel_calls=tf.data.AUTOTUNE,
            deterministic=False,
        )
        return ds.unbatch().prefetch(tf.data.AUTOTUNE)

    def sample(self, seeds: tf.Tensor) -> tfgnn.GraphTensor:
        # seeds = tf.cast(seeds, tf.int32)
        batch_size = tf.size(seeds)
        # print(f"batch_size={batch_size}")
        seeds_ragged = tf.RaggedTensor.from_row_lengths(seeds, tf.ones([batch_size], dtype=tf.int64))
        return self._sampling_model(seeds_ragged)

In [10]:
train_ds_provider = SubgraphDatasetProvider(graph_tensor, train_sampling_sizes, tf.data.Dataset.from_tensor_slices(train_idx))
valid_ds_provider = SubgraphDatasetProvider(graph_tensor, train_sampling_sizes, tf.data.Dataset.from_tensor_slices(valid_idx))

example_input_graph_spec = train_ds_provider.input_graph_spec._unbatch()

In [11]:
if tf.config.list_physical_devices("TPU"):
  print(f"Using TPUStrategy")
  min_nodes_per_component = {"paper": 1}
  strategy = runner.TPUStrategy("local")
  train_padding = runner.FitOrSkipPadding(example_input_graph_spec, train_ds_provider, min_nodes_per_component)
  valid_padding = runner.TightPadding(example_input_graph_spec, valid_ds_provider, min_nodes_per_component)
elif tf.config.list_physical_devices("GPU"):
  print(f"Using MirroredStrategy for GPUs")
  gpu_list = !nvidia-smi -L
  print("\n".join(gpu_list))
  strategy = tf.distribute.MirroredStrategy()
  train_padding = None
  valid_padding = None
else:
  print(f"Using default strategy")
  strategy = tf.distribute.get_strategy()
  train_padding = None
  valid_padding = None
print(f"Found {strategy.num_replicas_in_sync} replicas in sync")

Using MirroredStrategy for GPUs
GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-b2dd31a4-1d4c-6d81-647d-e36ea0e64af9)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Found 1 replicas in sync


In [12]:
def process_node_features(node_set: tfgnn.NodeSet, node_set_name: str):
    if node_set_name == "product":
        return {"feature": node_set["feature"], "label": node_set["label"]}
    raise KeyError(f"Unexpected node_set_name='{node_set_name}'")

def drop_all_features(_, **unused_kwargs):
    return {}

process_features = tfgnn.keras.layers.MapFeatures(
    context_fn=drop_all_features,
    node_sets_fn=process_node_features,
    edge_sets_fn=drop_all_features,
)

In [13]:
add_readout = tfgnn.keras.layers.AddReadoutFromFirstNode("seed", node_set_name="product")
move_label_to_readout = tfgnn.keras.layers.StructuredReadoutIntoFeature(
    "seed", feature_name="label", new_feature_name="category", remove_input_feature=True
)

In [14]:
feature_processors = [
    process_features,
    add_readout,
    move_label_to_readout,
]

In [15]:
node_state_dim = 128

def set_initial_node_states(node_set: tfgnn.NodeSet, node_set_name: str):
    if node_set_name == "product":
        return tf.keras.layers.Dense(node_state_dim, "relu")(node_set["feature"])
    raise KeyError(f"Unexpected node_set_name='{node_set_name}'")

In [16]:
num_graph_updates = 1
message_dim = 128
state_dropout_rate = 0.2
l2_regularization= 1e-5

def model_fn(graph_tensor_spec: tfgnn.GraphTensorSpec):
    graph = inputs = tf.keras.layers.Input(type_spec=graph_tensor_spec)
    graph = tfgnn.keras.layers.MapFeatures(
        node_sets_fn = set_initial_node_states)(graph)
    for i in range(num_graph_updates):
        graph = mt_albis.MtAlbisGraphUpdate(
            units = node_state_dim,
            message_dim = message_dim,
            receiver_tag = tfgnn.SOURCE,
            node_set_names = None if i < num_graph_updates - 1 else ["product"],
            simple_conv_reduce_type="mean|sum",
            state_dropout_rate=state_dropout_rate,
            l2_regularization=l2_regularization,
            normalization_type="layer",
            next_state_type="residual",
        )(graph)
    return tf.keras.Model(inputs, graph)

In [17]:
# np.unique(label.flatten()).shape gives 47
task = runner.NodeMulticlassClassification(
    num_classes=47,
    label_feature_name="category",
)

In [18]:
global_batch_size = 128
epochs = 30
initial_learning_rate = 0.001

if tf.config.list_physical_devices("TPU"):
    epoch_divisor = 1
else:
    epoch_divisor = 1

steps_per_epoch = NUM_TRAINING_SAMPLES // global_batch_size // epoch_divisor
validation_steps = NUM_VALIDATION_SAMPLES // global_batch_size // epoch_divisor

learning_rate = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate, steps_per_epoch*epochs)
optimizer_fn = functools.partial(tf.keras.optimizers.Adam, learning_rate=learning_rate)

trainer = runner.KerasTrainer(
    strategy=strategy,
    model_dir="/tmp/gnn_model",
    callbacks=[TensorBoard(log_dir="logs")],
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    restore_best_weights=False,
    checkpoint_every_n_steps="never",
    summarize_every_n_steps="never",
    backup_and_restore=False
)

In [19]:
model_exporter = runner.KerasModelExporter(output_names="product_category")
runner.run(
    gtspec=example_input_graph_spec,
    train_ds_provider=train_ds_provider,
    train_padding=train_padding,
    # valid_ds_provider=None,
    valid_ds_provider=valid_ds_provider,
    valid_padding=valid_padding,
    global_batch_size=global_batch_size,
    epochs=epochs,
    feature_processors=feature_processors,
    model_fn=model_fn,
    task=task,
    optimizer_fn=optimizer_fn,
    trainer=trainer,
    model_exporters=[model_exporter],
)

Epoch 1/30
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tens

2025-11-23 14:16:24.179644: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-11-23 14:16:24.213582: I external/local_xla/xla/service/service.cc:163] XLA service 0x79b01898dd20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-23 14:16:24.213592: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-11-23 14:16:24.222185: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1763907384.292899    9319 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1536/1536 [==============================] - 11s 6ms/step - loss: 0.7417 - sparse_categorical_accuracy: 0.7952 - sparse_categorical_crossentropy: 0.7374 - val_loss: 0.5436 - val_sparse_categorical_accuracy: 0.8523 - val_sparse_categorical_crossentropy: 0.5387
Epoch 2/30
1536/1536 [==============================] - 9s 6ms/step - loss: 0.5018 - sparse_categorical_accuracy: 0.8601 - sparse_categorical_crossentropy: 0.4963 - val_loss: 0.4907 - val_sparse_categorical_accuracy: 0.8666 - val_sparse_categorical_crossentropy: 0.4846
Epoch 3/30
1536/1536 [==============================] - 9s 6ms/step - loss: 0.4541 - sparse_categorical_accuracy: 0.8729 - sparse_categorical_crossentropy: 0.4474 - val_loss: 0.4706 - val_sparse_categorical_accuracy: 0.8718 - val_sparse_categorical_crossentropy: 0.4632
Epoch 4/30
1536/1536 [==============================] - 9s 6ms/step - loss: 0.4244 - sparse_categorical_accuracy: 0.8812 - sparse_categorical_crossentropy: 0.4164 - val_loss: 0.4599 - val_sparse_categ

INFO:tensorflow:Assets written to: /tmp/gnn_model/export/assets


RunResult(preprocess_model=<tf_keras.src.engine.functional.Functional object at 0x79b0b43dc2f0>, base_model=<tf_keras.src.engine.sequential.Sequential object at 0x79b0401f9e50>, trained_model=<tf_keras.src.engine.functional.Functional object at 0x79b15810d010>)

In [29]:
saved_model = tf.saved_model.load(os.path.join(trainer.model_dir, "export"))
signature_fn = saved_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

def _clean_example_for_serving(graph_tensor):
    graph_tensor = graph_tensor.remove_features(node_sets={"product": ["label"]})
    serialized_example = tfgnn.write_example(graph_tensor)
    return serialized_example.SerializeToString()

num_examples = 10
demo_ds = valid_ds_provider.get_dataset(tf.distribute.InputContext())
dds = itertools.islice(demo_ds, num_examples)
serialized_examples = [_clean_example_for_serving(gt) for gt in dds]

ds = tf.data.Dataset.from_tensor_slices(serialized_examples)
input_dict = {"examples": next(iter(ds.batch(num_examples)))}
output_dict = signature_fn(**input_dict)
logits = output_dict["product_category"]
probabilities = tf.math.softmax(logits).numpy()
classes = probabilities.argmax(axis = 1)
for i, c in enumerate(classes):
    print(f"The predicted class for input {i} is {c:3} "
          f"with predicted probability {probabilities[i, c]:.4}")

The predicted class for input 0 is   0 with predicted probability 0.9607
The predicted class for input 1 is  13 with predicted probability 0.9999
The predicted class for input 2 is   3 with predicted probability 0.9308
The predicted class for input 3 is   4 with predicted probability 0.9997
The predicted class for input 4 is  13 with predicted probability 0.9926
The predicted class for input 5 is   3 with predicted probability 0.5268
The predicted class for input 6 is   7 with predicted probability 0.9997
The predicted class for input 7 is   4 with predicted probability 1.0
The predicted class for input 8 is  10 with predicted probability 0.9874
The predicted class for input 9 is   8 with predicted probability 0.9993
